Introduction to PyTorch

Simple neural network example
that classify the Iris flower dataset.

I have put the iris.csv dataset, which contains 150 x 5 data and 3 different classes, in Chapter01 file

Connect your google colab runtime to your Google Drive,

In [0]:
from google.colab import drive
drive.mount('/content/drive') # do not use any space to indicate name of a file

Set current directory to necessary path,

In [2]:
cd drive/My\ Drive/Machine\ Learning

/content/drive/My Drive/Machine Learning


Check current directory,

In [3]:
pwd

'/content/drive/My Drive/Machine Learning'

Check what we have,

In [0]:
ls

Clone the repository to current file if it does not exit there,

In [0]:
!git clone https://github.com/OnurcanKoken/Python-Deep-Learning-Second-Edition

Go to Chapter01 file

In [5]:
cd Python-Deep-Learning-Second-Edition/Chapter01/

/content/drive/My Drive/Machine Learning/Python-Deep-Learning-Second-Edition/Chapter01


Classify the Iris flower dataset,

The original code gives an error here, pd.read_csv() gets an .csv format dataset but there exist .data file, so I downloaded and changed file format as .csv

In [0]:
import pandas as pd

dataset = pd.read_csv('iris.csv',
                      names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])
# originally; 
# dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
#                       names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])



In [0]:
dataset['species'] = pd.Categorical(dataset['species']).codes

dataset = dataset.sample(frac=1, random_state=1234)

# flower properties/flower class
# 120 samples for training
# 30 samples for testing

train_input = dataset.values[:120, :4]
train_target = dataset.values[:120, 4]

test_input = dataset.values[120:, :4]
test_target = dataset.values[120:, 4]

Feedforward network with one hidden layer with five units,

A ReLU activation function

An output layer with three units


*   Has three units
*   Each unit corresponds to one of the three classes of Iris flower

One-hot encoding for the target data will be used which means each class of the flower will be represented as an array

Iris Setosa = [1, 0, 0]; Iris Versicolour = [0, 1, 0]; Iris Virginica = [0, 0, 1]

One element of the array will be the target for one unit of the output layer. When the network classifies a new sample, we'll determine the class by taking the unit with the highest activation value.



In [0]:
# define first neural network

import torch

# enables us to use the same random data 
# every time for the reproducibility of results
torch.manual_seed(1234)

hidden_units = 5

net = torch.nn.Sequential(
    torch.nn.Linear(4, hidden_units),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_units, 3)
)

Loss funtion will measure how different the output of the network is compared to the target data.

Stochastic Gradient Descent optimizer with a learning rate of 0.1 and a momentum of 0.9. The SGD is a variation of the gradient descent algorithm.

In [0]:
# choose optimizer and loss function
criterion = torch.nn.CrossEntropyLoss() # loss function
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)


In [10]:
# train
epochs = 50 # iterate 50 times over the training dataset

for epoch in range(epochs):
    # create the torch variable from numpy arrays
    inputs = torch.autograd.Variable(torch.Tensor(train_input).float())
    targets = torch.autograd.Variable(torch.Tensor(train_target).long())

    # to prevent accumulation from the previous iterations
    # since we feed the training data to the neural network net (input)
    # and we compute the loss funtion criterion(out, target)
    # between the network output and the target data
    optimizer.zero_grad() 
    # propagete the loss value back through the network
    # so that we can calculate 
    # how each network weight affects the loss function
    out = net(inputs)
    loss = criterion(out, targets)
    loss.backward()
    # optimizer updates the weights of the network
    # to reduce the future loss function values
    optimizer.step() 

    if epoch == 0 or (epoch + 1) % 10 == 0:
        print('Epoch %d Loss: %.4f' % (epoch + 1, loss.data))
# originally;
#   if epoch == 0 or (epoch + 1) % 10 == 0:
#       print('Epoch %d Loss: %.4f' % (epoch + 1, loss.data[0]))
# I have changed because: https://stackoverflow.com/questions/56483122/indexerror-invalid-index-of-a-0-dim-tensor-use-tensor-item-to-convert-a-0-di

Epoch 1 Loss: 1.2181
Epoch 10 Loss: 0.6745
Epoch 20 Loss: 0.2447
Epoch 30 Loss: 0.1397
Epoch 40 Loss: 0.1001
Epoch 50 Loss: 0.0855


Final Accuracy of our model

Feeding the test set to the network and computing the error manually,

In [11]:

# test
import numpy as np

inputs = torch.autograd.Variable(torch.Tensor(test_input).float())
targets = torch.autograd.Variable(torch.Tensor(test_target).long())

optimizer.zero_grad()
out = net(inputs)
_, predicted = torch.max(out.data, 1)

error_count = test_target.size - np.count_nonzero((targets == predicted).numpy())
print('Errors: %d; Accuracy: %d%%' % (error_count, 100 * torch.sum(targets == predicted) / test_target.size))


Errors: 0; Accuracy: 100%


We were able to classify only 30 test samples and all correct.

We must keep in mind trying different hyperparameters of the network and see how the accuracy and loss function work.
We can try to change the number of units in the hidden layer, the number of epochs we train in the network, as well as the learning rate.